In [ ]:
!pip install google-cloud-bigquery google-cloud-aiplatform --quiet

In [ ]:
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

project_id = "qwiklabs-gcp-01-f5f87dad8c41"
location = "us-central1"

vertexai.init(project=project_id, location=location)
bq_client = bigquery.Client(project=project_id)

print("Initialization complete")

Initialization complete


In [ ]:
def retrieve_context(question):
    query = f"""
    SELECT base.string_field_0, base.string_field_1
    FROM VECTOR_SEARCH(
      TABLE `aurora_rag.aurora_faqs_embedded`,
      'embedding',
      (
        SELECT
          ml_generate_embedding_result AS embedding
        FROM ML.GENERATE_EMBEDDING(
          MODEL `aurora_rag.embedding_model`,
          (SELECT '{question}' AS content)
        )
      ),
      top_k => 3
    )
    """

    results = bq_client.query(query).result()

    context = ""
    for row in results:
        context += f"Q: {row.string_field_0}\nA: {row.string_field_1}\n\n"

    return context

In [ ]:
model = GenerativeModel("gemini-2.0-flash-001")

def ask_chatbot(question):
    context = retrieve_context(question)

    print("=== Retrieved Context ===")
    print(context)

    prompt = f"""
    You are a helpful assistant for Aurora Bay.

    Use ONLY the information provided below to answer the question.
    If the answer is not in the information, say you don't know.

    {context}

    Question: {question}
    """

    response = model.generate_content(prompt)
    return response.text

In [ ]:
print(ask_chatbot("Where is the Aurora Bay Town Hall located?"))

=== Retrieved Context ===
Q: How can I apply for a business license in Aurora Bay?
A: Applications for a business license can be submitted either online via the town’s official website or in person at the Town Hall. Processing takes around 2–3 weeks.

Q: What is the population of Aurora Bay?
A: Aurora Bay has a population of approximately 3,200 residents, although it can fluctuate seasonally due to temporary fishing and tourism workforces.

Q: What educational facilities are available for children?
A: Aurora Bay has one elementary school, one middle school, and a combined high school. They’re managed by the Aurora Bay School District.


I don't know.

